Missing:
    
* Apply Random Forest
* Test, Train
* Results
   
* Compare with HashingVectorizer

# Sentiment prediction on IMDB reviews


25,000 IMDB movie reviews, each with a positive or negative sentiment label.

The reviews have html tags and we will use BeautifulSoup to remove the tags


In [7]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re

df = pd.read_csv("../data/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
print(df.columns)
print(df.shape[0])
# Inspect the first review
print(df["review"][0])


Index(['id', 'sentiment', 'review'], dtype='object')
25000
"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.

In [8]:
# remove html tags
example1 = BeautifulSoup(df["review"][0], "html.parser")

print(example1.get_text())


"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

# Remove HTML tags

In [11]:
df['clean_text'] = df.review.apply(lambda x : BeautifulSoup(x, "html.parser").get_text() )


In [12]:
# check a few reviews 
df['clean_text'][2]


'"The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, and some scientists resurrect one of nature\'s most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack of large pre-historical animals which are deadlier and bigger . In addition , a security agent (Stacy Haiduk) and her mate (Brian Wimmer) fight hardly against the carnivorous Smilodons. The Sabretooths, themselves , of course, are the real star stars and they are astounding terrifyingly though not convincing. The giant animals savagely are stalking its prey and the group run afoul and fight against 

# Vectorize 

Apply vectorizer = CountVectorizer  with 

* max_features = 5000
* stop_words = 'english'
* analyzer = "word"

to df.clean_text

And look at the vocabulary of the vectorizer




In [21]:
vectorizer = CountVectorizer( analyzer = "word",
                             stop_words = 'english',
                             max_features = 5000) 

df['text'] = vectorizer.fit_transform(df.clean_text)
# vectorizer.vocabulary_

# Random Forest

from sklearn.ensemble import RandomForestClassifier

* Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

* Fit the forest to the training set, using the bag of words as  features and the sentiment labels as the response variable



In [24]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100) 

forest.fit( df.text, df.sentiment )

ValueError: setting an array element with a sequence.